In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, TensorDataset

from llama import Tokenizer
tokenizer = Tokenizer('./tokenizer.model')

#hyperparameters
device = 'cuda' if torch.cuda.is_available() else 'cpu'
batch_size = 16
block_size = 512
eval_iters = 100
n_head = 4
n_layer = 4
n_embd = 128
learning_rate = 1e-3
vocab_size = tokenizer.n_words

text = open('../data/shakespeare.txt', encoding="utf-8").read()

data = torch.tensor(tokenizer.encode(text, False, False)).to(device)
n = int(len(data)*0.9)
train_data = data[:n]
val_data = data[n:]

def get_all_data(split):
    
    data = train_data if split == 'train' else val_data
    
    X = torch.stack([data[i:i+block_size] for i in range(len(data) - block_size)])
    y = torch.stack([data[i+1:i+1+block_size] for i in range(len(data) - block_size)])
    
    X = X.to(device)
    y = y.to(device)
    
    return X, y

def get_batch(split):
    
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    X = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+1+block_size] for i in ix])
    X, y = X.to(device), y.to(device)
    return X, y

@torch.no_grad()
def estmiate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            xb, yb = get_batch(split)
            logits, loss = model(xb, yb)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

class Head(nn.Module):
    
    """single head of self attention"""
    
    def __init__(self, head_size):
        super().__init__()
        
        self.key = nn.Linear(n_embd, head_size, bias=False)
        self.query = nn.Linear(n_embd, head_size, bias=False)
        self.value = nn.Linear(n_embd, head_size, bias=False)
        self.register_buffer('tril', torch.tril(torch.ones(block_size, block_size)))
        
    def forward(self, x):
        
        B, T, C = x.shape
        
        k = self.key(x)
        q = self.query(x)
        
        wei = q @ k.transpose(-2, -1)
        wei = wei.masked_fill(self.tril[:T, :T] == 0, float('-inf'))
        wei = F.softmax(wei, dim=-1)
        
        v = self.value(x)
        
        out = wei @ v
        
        return out
    
class MultiHeadAttention(nn.Module):
    
    def __init__(self, head_size):
        super().__init__()
        
        self.heads = nn.ModuleList([Head(head_size) for _ in range(n_head)])
        self.proj = nn.Linear(n_embd, n_embd)
        
    def forward(self, x):
        
        x = torch.cat([h(x) for h in self.heads], dim=-1)
        x = self.proj(x)
        return x
    

class FeedForward(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.net = nn.Sequential(
            nn.Linear(n_embd, 4*n_embd),
            nn.ReLU(),
            nn.Linear(4*n_embd, n_embd)
        )
        
    def forward(self, x):
        return self.net(x)
    

class Block(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        head_size = n_embd // n_head
        
        self.sa = MultiHeadAttention(head_size)
        self.ffwd = FeedForward()
        self.ln1 = nn.LayerNorm(n_embd)
        self.ln2 = nn.LayerNorm(n_embd)
        
    def forward(self, x):
        
        x = x + self.ln1(self.sa(x))
        x = x + self.ln2(self.ffwd(x))
        
        return x
    
    
class Transformer(nn.Module):
    
    def __init__(self):
        super().__init__()
        
        self.token_embedding_table = nn.Embedding(vocab_size, n_embd)
        self.position_embedding_table = nn.Embedding(block_size, n_embd)
        self.blocks = nn.Sequential(*[Block() for _ in range(n_layer)])
        self.ln_f = nn.LayerNorm(n_embd)
        self.lm_head = nn.Linear(n_embd, vocab_size)
        
    def forward(self, idx, targets=None):
        
        B, T = idx.shape
        
        tok_emb = self.token_embedding_table(idx)
        pos_emb = self.position_embedding_table(torch.arange(T).to(device))
        x = tok_emb + pos_emb
        x = self.blocks(x)
        x = self.ln_f(x)
        logits = self.lm_head(x)
        
        if targets == None:
            loss = None
            
        else:
            
            B,T,C = logits.shape
            
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            
            loss = F.cross_entropy(logits, targets)
            
        return logits, loss
    
    def generate(self, idx, max_gen_len):
        
        model.eval()
        
        outputs = torch.cat((idx, torch.zeros(max_gen_len).unsqueeze(0).to(device)), dim=-1)
        
        outputs = outputs.squeeze(0)
        
        for i in range(len(idx[0]), max_gen_len + len(idx[0])):
        
            logits, loss = self(idx)

            logits = logits[:, -1, :]

            probs = F.softmax(logits, dim=-1)
            
            pred = torch.multinomial(probs, 1)
            
            outputs[i] = pred[0]
            
            idx = torch.cat((idx, pred), dim=-1)
        
        print(tokenizer.decode(outputs.long().tolist()))
    
    
X, y = get_all_data('train')
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size = batch_size, shuffle=True)
    

model = Transformer().to(device)
print(sum(p.numel() for p in model.parameters())/1e6, ' M parameters')

optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

count = 0
max_iters = 100

for xb, yb in dataloader:
    
    logits, loss = model(xb, yb)
    
    if count % (max_iters / 10) == 0:
        print(loss.item())
    
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
    
    if count == max_iters:
        break
        
    count += 1

context = torch.tensor(tokenizer.encode("CLARK:", False, False)).unsqueeze(0).to(device)
model.generate(context, 500)

9.081344  M parameters
10.516077995300293
8.485395431518555
7.145170211791992
6.47765588760376
6.193207740783691
6.137439250946045
5.953705787658691
5.808291912078857
5.787979602813721
5.820441246032715
5.638680934906006


In [7]:
dir(tokenizer)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 'bos_id',
 'decode',
 'encode',
 'eos_id',
 'n_words',
 'pad_id',
 'sp_model']